## Ch 1. Elements of Reinforcement Learning

### Policy
* A **policy** defines the agent's **way of behaving** at a **given time**.
* A policy is a **mapping** from perceived **states** of the environment to **actions** to be taken when in those states.
  - E.g., a simple function, a lookup table, or a search process. 
* The policy is the core of an agent (it alone is sufficient to determine behavior).
* Policies may be **stochastic**, specifying probabilities for each action.

### Reward signal
* On each time step, the environment sends to the agent a single number, called the **reward**.
* A reward signal defines the **goal** of a RL problem. 
* The agent's sole objective is to **maximize the total reward** it receives over the long run.
* The **reward signal**, thus, defines what are the good and bad events for the agent.
* In general, reward signals may be stochastic functions of the state of the environment and the actions taken.

### Value function
* A **value function** specifies what is good in the **long run**, whereas the reward signal indicates what is good in an **immediate sense**.
* The **value of a state** is the **total amount of reward** an agent can expect to accumulate over the future, starting from that state.
* It is values with which we are most concerned when making and evaluating decisions (action choices are made based on value judgments).
  - We seek actions that bring about states of highest value, not highest reward.

### Model
* A **model** is something that mimics the **behavior of the environment** that allows inferences to be made about how the environment will behave.
  - E.g., given a state and action, the model might predict the resultant next state and next reward.
* Models are used for **planning**, by which we mean any way of deciding on a course of action by considering possible future situations before they are actually experienced.
* Methods for solving RL problems that use models and planning are called **model-based** methods, as opposed to simpler **model-free** methods that are explicitly **trial-and-error** learners (viewed as almost the opposite of planning).

---
# Part I: Tabular Solution Methods
Here the **state and action spaces** are **small** enough for the approximate **value functions** to be represented as arrays, or **tables**. In this case, the methods can often find **exact solutions**, that is, they can often find exactly the **optimal value function** and the **optimal policy**.

## Ch 2. Multi-armed Bandits
* The most important feature distinguishing RL from other types of learning is that it uses training information that **evaluates** the actions taken rather than **instructs** by giving correct actions. 
  - Purely **evaluative** feedback indicates how good the action taken was, but not whether it was the best or the worst action possible.
  - Purely **instructive** feedback indicates the correct action to take, independently of the action actually taken (this kind of feedback is the basis of supervised learning).
  - These two kinds of feedbacks (in their pure forms) are quite distinct: evaluative feedback depends entirely on the action taken, whereas instructive feedback is independent of the action taken.

### k-armed Bandit Problem
* You are faced repeatedly with a choice among **k different options (actions)**.
* After each choice you receive a numerical **reward** chosen from a stationary probability distribution that depends on the action you selected.
* Your **objective** is to **maximize the expected total reward** over some time period (e.g., over 1000 action selections), or **time steps**.
* Each action has an **expected or mean reward** given that that action is selected (it's called the **value of that action**).
* If we denote the action selected on time step $t$ as $A_t$, and the corresponding reward as $R_t$, then the value of an arbitrary action $a$, denoted $q_{*}(a)$, is the expected reward given that a is selected:
$$
q_{*}(a) = \mathbb{E}[R_t | A_t=a]
$$
* If you knew the value of each action, then it would be trivial to solve the k-armed bandit problem: you would always select the action with highest value. 
* If we do not know the action values with certainty, we need to use the **estimated value of action** a at time step $t$, denoted by $Q_t(a)$. We would like $Q_t(a)$ to be close to $q_{*}(a)$.
* **Greedy actions**: any action whose estimated value is greatest at any time step
* **Exploiting**: if we select one of the **greedy actions**, then we say that we are exploiting our current knowledge of the values of the actions.
* **Exploring**: if we select one of the **nongreedy actions**, then we say we are exploring, because this enables us to improve our estimate of the nongreedy action's value.
* **Exploitation** is the right thing to do to maximize the expected reward on the **one step**, but **exploration** may produce the greater total reward in the **long run**. 
* Because it is not possible both to explore and to exploit with any single action selection, one often refers to the **conflict** between exploration and exploitation.

### Action-value Methods
* We collectively call **action-value methods** to **estimate the values of actions** and use the estimates to make **action selection** decisions.
* The **true value of an action** is the **mean reward** when that action is selected. One natural way to estimate this is by averaging the rewards actually received (it's called the **sample-average** method for estimating action values):
$$
Q_t(a) = \frac{\text{sum of rewards when }a\text{ taken prior to }t}{\text{number of times }a\text{ taken prior to }t} = \frac{\sum_{i=1}^{t-1}R_i\cdot\mathbb{1}_{A_i=a}}{\sum_{i=1}^{t-1}\mathbb{1}_{A_i=a}}
$$
  - As the denominator goes to **infinity**, by the law of large numbers, $Q_t(a)$ converges to $q_*(a)$.
* The **greedy action selection** method can be written as below:
$$
A_t = \arg\max_aQ_t(a)
$$
* **$\varepsilon$-greedy methods**: behave greedily most of the time, but every once in a while (with small probability), select **randomly** from among all the actions with equal probability, independently of the action-value estimates. 
  - An advantage of these methods is that, in the limit as the number of steps increases, every action will be sampled an infinite number of times, thus ensuring that all the $Q_t(a)$ converge to $q_*(a)$.

### Incremental Implementation
* The above action-value methods estimate action values as sample averages of **observed rewards**.
* Let $R_i$ denotes the **reward** received after the $i$th selection of this action, and $Q_n$ denotes the **estimate of its action-value** after it has been selected $n-1$ times:
$$
Q_n = \frac{R_1 + R_2 + \cdots + R_{n-1}}{n-1}
$$
* The obvious implementation would be to maintain a record of all the rewards and then perform this computation whenever the estimated value was needed. 
* How these averages can be computed with **constant memory** and **constant per-time-step computation**?
$$
\begin{equation}
\begin{aligned}
Q_{n+1} ={} & \frac{1}{n}\sum_{i=1}^nR_i\\
={} & \frac{1}{n}(R_n + \sum_{i=1}^{n-1}R_i)\\
={} & \frac{1}{n}(R_n + (n-1)\frac{1}{n-1}\sum_{i=1}^{n-1}R_i)\\
={} & \frac{1}{n}(R_n + (n-1)Q_n)\\
={} & Q_n + \frac{1}{n}(R_n - Q_n)\\
\end{aligned}
\end{equation}
$$
* The general form
$$
NewEstimate \leftarrow OldEstimate + StepSize\cdot[Target - OldEstimate]
$$
* $[Target - OldEstimate]$ is and **error** in the estimate.
<img src="rl_figs/rl_simple_bandit.png" width="700">

### Nonstationary Problem
* The averaging methods discussed so far are appropriate for **stationary bandit problems**, that is, for bandit problems in which the **reward probabilities do not change over time**.
* In nonstationary cases it makes sense to **give more weight to recent rewards** than to long-past rewards.
* One of the most popular ways of doing this is to use a **constant step-size parameter**.

$$
Q_{n+1} = Q_n + \alpha(R_n - Q_n)
$$

* where the step-size parameter $\alpha \in (0, 1]$ is constant.

$$
\begin{equation}
\begin{aligned}
Q_{n+1} ={} & \alpha(R_n - Q_n)\\
={} & \cdots\\
={} & (1 - \alpha)^nQ1 + \sum_{i=1}^n \alpha(1 - \alpha)^{n-i}R_i\\
\end{aligned}
\end{equation}
$$

* We call this a weighted average because the sum of the weights is $(1 - \alpha)^n + \sum_{i=1}^n \alpha(1 - \alpha)^{n-i} = 1$.
* The quantity $1 - \alpha$ is less than 1, and thus the weight given to $R_i$ decreases as the number of intervening rewards increases.

---
## Ch 3. Finite Markov Decision Processes
* Finite Markov Decision Processes (MDP) are a classical formalization of sequential decision making, where actions influence not just immediate rewards, but also **subsequent situations**, or **states**, and through those future rewards.
* Thus, MDPs involve **delayed reward** and the need to tradeoff immediate and delayed reward.
* Whereas in **bandit problems** we estimated the value $q_*(a)$ of each action $a$, in **MDPs** we estimate the value $q_*(s, a)$ of each action $a$ in each state $s$, or we estimate the value $v_*(s)$ of each state given optimal action selections.
* These **state-dependent quantities** are essential to accurately assigning credit for **long-term** consequences to individual action selections.

### The Agent–Environment Interface
* MDPs are meant to be a straightforward framing of the problem of **learning from interaction** to achieve a goal.
* The **learner and decision maker** is called the **agent**. 
* The thing an agent interacts with (comprising everything outside the agent) is called the **environment**.
* These interact **continually**, the agent selecting actions and the environment responding to these actions and presenting new situations to the agent.
<img src="rl_figs/rl_agent_env.png" width="400">
* The agent and environment interact at each of a sequence of discrete time steps, $t = 0, 1, 2, 3, \cdots$. 
  - At each time step $t$, the agent receives some **representation of the environment's state**, $S_t \in \mathcal{S}$, and on that basis selects an **action**, $A_t \in \mathcal{A}(s)$.
  - One time step later, in part as a consequence of its action, the agent receives a **numerical reward**, $R_{t+1} \in \mathcal{R} \subset \mathbb{R}$, and finds itself in a new state, $S_{t+1}$.
  - The MDP and agent together thereby give rise to a sequence or **trajectory** that begins like this: $S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3, \cdots$.
  
  
* In a **finite** MDP, the sets of states, actions, and rewards ($\mathcal{S}$, $\mathcal{A}$, and $\mathcal{R}$) all have a **finite number of elements**.
  - In this case, the random variables $R_t$ and $S_t$ have well defined discrete probability distributions dependent only on the **preceding state and action**. 
  - That is, for particular values of these random variables, $s' \in \mathcal{S}$ and $r \in \mathcal{R}$, there is a probability of those values occurring at time $t$, given particular values of the preceding state and action (for all $s',s \in \mathcal{S}$, $r \in \mathcal{R}$, and $a \in \mathcal{A}(s)$):
$$
p(s',r|s,a) = Pr\{S_t=s',R_t=r|S_{t-1}=s,A_{t-1}=a\}
$$
* The function $p$ defines the **dynamics** of the MDP: $p:\mathcal{S} \times \mathcal{R} \times \mathcal{S} \times \mathcal{A} \rightarrow [0, 1]$.
* **State-transition** probabilities: $p:\mathcal{S} \times \mathcal{S} \times \mathcal{A} \rightarrow [0, 1]$.
$$
p(s'|s,a) = Pr\{S_t=s'|S_{t-1}=s,A_{t-1}=a\} = \sum_{r \in \mathcal{R}}p(s',r|s,a)
$$
* **State-action** pairs: $r: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$.
$$
r(s,a) = \mathbb{E}[R_t|S_{t-1}=s,A_{t-1}=a\} = \sum_{r \in \mathcal{R}}r\sum_{s' \in \mathcal{S}}p(s',r|s,a)
$$
* **State-action-next-state** pairs: $r: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}$.
$$
r(s,a,s') = \mathbb{E}[R_t|S_{t-1}=s,A_{t-1}=a,S_t=s'\} = \sum_{r \in \mathcal{R}}r\frac{p(s',r|s,a)}{p(s'|s,a)}
$$


* In a **Markov decision process**, the probabilities given by $p$ completely characterize the **environment's dynamics**. 
  - That is, the probability of each possible value for $S_t$ and $R_t$ depends only on the **immediately preceding state and action**, $S_{t-1}$ and $A_{t-1}$.
  - This is best viewed a **restriction** not on the decision process, but on the **state**. 
  - The state must include information about all aspects of the past agent-environment interaction that make a difference for the future. 
  - If it does, then the state is said to have the **Markov property**.

### Goals, Rewards, Returns and Episodes
* In RL, the purpose or **goal** of the agent is formalized in terms of a special signal, called the **reward**, passing from the environment to the agent. 
  - At each time step, the reward is a **simple number**, $R_t \in \mathbb{R}$.
* Informally, the agent's goal is to **maximize** the total amount of reward it receives.
  - This means maximizing not immediate reward, but **cumulative reward** in the **long run**.
  
#### Episodic tasks
* If the sequence of rewards received after time step $t$ is denoted $R_{t+1}$, $R_{t+2}$, $R_{t+3}$, $\cdots$, then what precise aspect of this sequence do we wish to maximize?
* In general, we seek to maximize the **expected return**, denoted $G_t$, which is defined as some specific function of the reward sequence.
* The **return** is the function of future rewards that the agent seeks to maximize (in expected value).
* In the simplest case the **return** is the **sum of the rewards** ($T$ is a final time step):
$$
G_t = R_{t+1} + R_{t+2} + R_{t+3} + \cdots + R_T
$$
* The above approach makes sense where there is a natural notion of **final time step**, e.g., $T$.
* That is, when the agent-environment interaction breaks naturally into subsequences, called **episodes**.
* Each episode ends in a special state, called the **terminal state**, followed by a reset to a standard starting state or to a sample from a standard distribution of starting states.
* Even if you think of episodes as ending in different ways, such as winning and losing a game, the next episode begins **independently** of how the previous one ended. 
* Thus, the episodes can all be considered to end in the **same terminal state**, with different rewards for the different outcomes. 
* Tasks with episodes of this kind are called **episodic tasks**.

#### Continuing tasks
* In many cases the agent-environment interaction does not break into episodes, but goes on **continually without limit**.
* The above formulation is problematic for continuing tasks because the final time step would be $T = \infty$, and the return, which is what we are trying to maximize, could itself easily be infinite.
* A concept that we need here is that of **discounting**.
* According to the discounting approach, the agent tries to select actions so that the **sum of the discounted rewards** it receives over the future is maximized. 
* In particular, it chooses $A_t$ to maximize the expected **discounted return** ($0 \le \gamma \le 1$ is called the **discount rate**):
$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}
$$
* It can also be written as
$$
\begin{equation}
\begin{aligned}
G_t ={} & R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 R_{t+4} + \cdots\\
={} & R_{t+1} + \gamma (R_{t+2} + \gamma R_{t+3} + \gamma^1 R_{t+4} + \cdots)\\
={} & R_{t+1} + \gamma G_{t+1}\\
\end{aligned}
\end{equation}
$$
* Although the return of the following formula is a sum of an infinite number of terms, it is still finite if the reward is nonzero and constant ($\gamma < 1$). For example, if the reward is a constant +1 ($R = +1$), then the return is:
$$
G_t = \sum_{k=0}^{\infty} \gamma^k = \frac{1}{1-\gamma}
$$

#### Unified notation for episodic and continuing tasks
* The **return** is defined as a sum over a **finite number** of terms in **episodic tasks**, and as a sum over an **infinite number** of terms in **continuous tasks**.
* These two can be **unified** by considering episode termination to be the entering of a special **absorbing state** that transitions only to itself and that generates only rewards of zero.

<img src="rl_figs/rl_absorbing_state.png" width="400">

### Policies and Value Functions
* Almost all RL algorithms involve estimating **value functions**: functions of **states** (or of **state-action** pairs) that estimate **how good** it is for the agent to **be in a given state** (or how good it is to **perform a given action in a given state**).
* The notion of "how good" here is defined in terms of **future rewards** that can be expected, or, to be precise, in terms of expected return. 
* The **rewards** the agent can expect to receive in the future **depend on what actions** it will take. Accordingly, **value functions** are defined with respect to **particular ways of acting**, called **policies**.
* Formally, a **policy** is a mapping from **states** to probabilities of selecting each possible **action**.
* If the agent is following policy $\pi$ at time $t$, then $\pi(a|s)$ is the probability that $A_t=a$, if $S_t=s$.

#### State-value function
* The **value function** of a state $s$ under a policy $\pi$, denoted $v_{\pi}(s)$, is the expected return when starting in $s$ and following $\pi$ thereafter. For MDPs, we can define $v_{\pi}$ formally by:
$$
v_{\pi}(s) = \mathbb{E}_{\pi}[G_t|S_t=s] = \mathbb{E}_{\pi}[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1}|S_t=s]\text{, for all } s \in \mathcal{S}
$$
* We call the function $v_\pi$ the **state-value function for policy $\pi$**.
* Note that the **value** of the **terminal state**, if any, is always **zero**.


#### Action-value function
* Similarly, we define the **value** of **taking action a in state** $s$ under a policy $\pi$, denoted $q_{\pi}(s, a)$, as the **expected return** starting from $s$, taking the action $a$, and thereafter following policy $\pi$:
$$
q_{\pi}(s, a) = \mathbb{E}_{\pi}[G_t|S_t=s,A_t=a] = \mathbb{E}_{\pi}[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1}|S_t=s,A_t=a]
$$
* We call the function $q_\pi$ the **action-value function for policy $\pi$**.

#### Estimating value functions
* The value functions $v_{\pi}$ and $q_{\pi}$ can be **estimated** from experience. 
  - For example, if an agent follows policy ${\pi}$ and maintains an **average** (for each **state** encountered) of the actual **returns** that have followed that state, then the average will converge to the **state's value**, $v_{\pi}(s)$, as the number of times that state is encountered approaches infinity.
  - If separate averages are kept for **each action** taken in **each state**, then these averages will similarly converge to the **action values**, $q_{\pi}(s, a)$.
* We call estimation methods of this kind **Monte Carlo methods** because they involve **averaging over many random samples** of actual returns.
* If there are very **many states**, then it may **not be practical** to keep **separate averages** for each state individually.
  - Instead, the agent would have to maintain $v_{\pi}$ and $q_{\pi}$ as **parameterized functions** (with fewer parameters than states) and adjust the parameters to better match the observed returns.
  
  
#### Bellman equation
* A fundamental property of value functions used throughout RL is that they satisfy **recursive relationships**. 
* For any policy $\pi$ and any state $s$, the following consistency condition holds between the value of $s$ and the value of its possible successor states:
$$
\begin{equation}
\begin{aligned}
v_{\pi}(s) ={} & \mathbb{E}_{\pi}[G_t|S_t=s]\\
={} & \mathbb{E}_{\pi}[R_{t+1} + \gamma G_{t+1}|S_t=s]\\
={} & \sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r + \gamma\mathbb{E}_{\pi}[G_{t+1}|S_{t+1}=s']]\\
={} & \sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]\\
\end{aligned}
\end{equation}
$$
* This equation is the **Bellman equation** for $v_{\pi}$, which expresses a relationship between the **value of a state** and the **values of its successor states**.
* Here is the **backup diagram** for $v_{\pi}$. The Bellman equation averages over all the possibilities, weighting each by its probability of occurring.
<img src="rl_figs/rl_backup_diag.png" width="250">

### Optimal Policies and Optimal Value Functions
* Solving a RL task means, roughly, **finding a policy** that achieves a lot of reward over the long run. 
* For **finite MDPs**, we can precisely define an optimal policy in the following way. 
  - Value functions define a partial ordering over policies. 
  - A policy $\pi$ is defined to be **better than or equal** to a policy $\pi'$, if its **expected return** is **greater than or equal** to that of $\pi'$ for all states ($\pi \ge \pi'$, if and only if $v_{\pi}(s) \ge v_{\pi'}(s)$ for all $s \in \mathcal{S}$).
* **Optimal policy**: there is always at least one policy that is better than or equal to all other policies. There may be more than one, we denote all the optimal policies by $\pi_*$.
* All optimal policies share the same **state-value function**, called the **optimal state-value function**, denoted $v_*$:
$$
v_*(s) = \max_{\pi}v_{\pi}(s),\text{ for all }s \in \mathcal{S}
$$

* Optimal policies also share the same **optimal action-value function**, denoted $q_*$, defined as:
$$
q_*(s,a) = \max_{\pi}q_{\pi}(s,a),\text{ for all }s \in \mathcal{S}\text{ and }a \in \mathcal{A}
$$
* We can write $q_*$ in terms of $v_*$ as follows:
$$
q_*(s,a) = \mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1})|S_t=s,A_t=a]
$$


---
## Ch 4. Dynamic Programming

* The term **dynamic programming (DP)** refers to a collection of algorithms that can be used to compute **optimal policies** given a **perfect model of the environment** as a **Markov decision process (MDP)**.
* The rest of the methods presented in this book can be viewed as attempts to achieve much the same effect as DP, only with less computation and without assuming a perfect model of the environment.
* We usually assume that the environment is a **finite MDP**.
  - Its state, action, and reward sets, $S$, $A$, and $R$, are finite
  - Its dynamics are given by a set of probabilities $p(s',r|s,a)$, for all $s \in S$, $a \in A(s)$, $r \in R$, and $s' \in S^+$ ($S^+$ is $S$ plus a terminal state if the problem is episodic).
* DP ideas can also be applied to problems with **continuous state and action spaces**, exact solutions are possible only in special cases.
  - A common way of obtaining approximate solutions for tasks with continuous states and actions is to **quantize the state and action spaces** and then apply finite-state DP methods.
* The key idea of DP (and of RL generally), is the use of **value functions** to organize and structure the search for **good policies**.
* As discussed before, we can obtain **optimal policies** once we have found the optimal **value functions**, $v_*(s)$ or $q_*(s, a)$, which satisfy the **Bellman optimality equations**.

$$
\begin{equation}
\begin{aligned}
v_*(s) ={} & \max_{a}\mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1})|S_t=s,A_t=a]\\
={} & \max_{a}\sum_{s',r}p(s',r|s,a)[r + \gamma v_*(s')]
\end{aligned}
\end{equation}
$$

$$
\begin{equation}
\begin{aligned}
q_*(s,a) ={} & \mathbb{E}[R_{t+1} + \gamma \max_{a'}q_*(S_{t+1}, a')|S_t=s,A_t=a]\\
={} & \sum_{s',r}p(s',r|s,a)[r + \gamma \max_{a'}q_*(s', a')]
\end{aligned}
\end{equation}
$$

### Policy Evaluation (Prediction)
* **Policy evaluation** refers to the computation of the **state-value function** $v_\pi$ for a given **policy** $\pi$.
$$
\begin{equation}
\begin{aligned}
v_{\pi}(s) ={} & \mathbb{E}_{\pi}[G_t|S_t=s]\\
={} & \mathbb{E}_{\pi}[R_{t+1} + \gamma G_{t+1}|S_t=s]\\
={} & \mathbb{E}_{\pi}[R_{t+1} + \gamma v_{\pi}(S_{t+1})|S_t=s]\\
={} & \sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]\\
\end{aligned}
\end{equation}
$$
* The existence and uniqueness of $v_\pi$ are guaranteed as long as either $\gamma < 1$ or eventual termination is guaranteed from all states under the policy $\pi$.
* Although its solution is a straightforward, iterative solution methods are most suitable.
  - Consider a sequence of **approximate value functions** $v_0, v_1, v_2, \cdots$, each mapping $\mathcal{S}^+$ to $\mathbb{R}$ (the real numbers).
  - The initial approximation, $v_0$ is chosen arbitrarily, and each successive approximation is obtained by using the **Bellman equation** for $v_\pi$ as an update rule:
$$
\begin{equation}
\begin{aligned}
v_{k+1}(s) ={} & \mathbb{E}_{\pi}[R_{t+1} + \gamma v_{k}(S_{t+1})|S_t=s]\\
={} & \sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r + \gamma v_{k}(s')]\\
\end{aligned}
\end{equation}
$$
* **Iterative policy evaluation**: the sequence $\{v_k\}$ can be shown in general to converge to $v_{\pi}$ as $k \rightarrow \infty$ under the same conditions.
* To produce each successive approximation ($v_{k+1}$ from $v_k$), **iterative policy evaluation** applies the same **operation** to each state $s$:
  - It replaces the old value of $s$ with a new value obtained from the old values of the successor states of $s$, and the expected immediate rewards, along all the one-step transitions possible under the policy being evaluated.
  - We call this kind of operation an **expected update**.
<img src="rl_figs/rl_policy_eval.png" width="700">

### Policy Improvement
* **Policy improvement** refers to the computation of an **improved policy** given the **value function** for that **policy**.
* Our reason for computing the **value function for a policy** is to help find **better policies**.
* Suppose we have determined the value function $v_\pi$ for an arbitrary deterministic policy $\pi$.
  - For some state $s$ we would like to know whether or not we should change the policy to deterministically choose an action $a \neq \pi(s)$.
  - We know how good it is to follow the current policy from $s$ (i.e., $v_\pi(s)$), but would it be better or worse to change to the new policy?
  - One way to answer this question is to consider selecting $a$ in $s$ and thereafter following the existing policy $\pi$.
$$
\begin{equation}
\begin{aligned}
q_{\pi}(s, a) ={} & \mathbb{E}_{\pi}[R_{t+1} + \gamma v_{\pi}(S_{t+1})|S_t=s,A_t=a]\\
={} & \sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]\\
\end{aligned}
\end{equation}
$$
  - If the above equation is greater than $v_{\pi}(s)$, then it means that it is better to select $a$ once in $s$ and thereafter follow $\pi$ all the time.
* **Policy improvement**: Let $\pi$ and $\pi'$ be any pair of deterministic policies such that, for all $s \in S$, $q_{\pi}(s, \pi'(s)) \ge v_{\pi}(s)$. Then the policy $\pi'$ must be as good as, or better than, $\pi$. That is, it must obtain greater or equal expected return from all states $s \in S$, $v_{\pi'}(s) \ge v_{\pi}(s)$.
* The process of making a **new policy** that **improves** on an **original policy**, by making it greedy with respect to the value function of the original policy, is called **policy improvement**.

### Policy Iteration
* Once a policy, $\pi$, has been improved using $v_\pi$ to yield a better policy, $\pi'$, we can then compute $v_\pi'$ and improve it again to yield an even better $v_\pi''$.
  - We thus obtain a **sequence** of monotonically improving policies and value functions:
<img src="rl_figs/rl_policy_iter2.png" width="500">
  - $E$ denotes a **policy evaluation**, and $I$ denotes a **policy improvement**.
* Each policy is guaranteed to be a strict improvement over the previous one (unless it is already optimal). 
* Because a finite MDP has only a finite number of policies, this process must converge to an optimal policy and optimal value function in a finite number of iterations.
<img src="rl_figs/rl_policy_iter.png" width="700">

### Value Iteration
* One **drawback** to **policy iteration** is that each of its iterations involves **policy evaluation**.
  - Each **policy evaluation** may itself be a protracted iterative computation requiring **multiple sweeps** through the state set.
  - If **policy evaluation** is done **iteratively**, then convergence exactly to $v_{\pi}$ occurs only in the limit.
* The **policy evaluation** step of **policy iteration** can be truncated in several ways without losing the convergence guarantees of policy iteration. 
* One important special case is when **policy evaluation** is stopped after **just one sweep** (one update of each state). This algorithm is called **value iteration**.
* The **value iteration** can be written as a simple update operation that combines the policy improvement and truncated policy evaluation steps:
$$
\begin{equation}
\begin{aligned}
v_{k+1}(s) ={} & \mathbb{E}_a[R_{t+1} + \gamma v_{k}(S_{t+1})|S_t=s,A_t=a]\\
={} & \max_a\sum_{s',r}p(s',r|s,a)[r + \gamma v_{k}(s')]\\
\end{aligned}
\end{equation}
$$
<img src="rl_figs/rl_value_iter.png" width="700">

### Generalized Policy Iteration
* **Policy iteration** consists of two simultaneous, interacting processes:
  1. **Policy-evaluation**: makes the **value function** consistent with the **current policy**.
  2. **Policy-improvement**: makes the **policy greedy** with respect to the **current value function**.
<img src="rl_figs/rl_gpi.png" width="170">
* In **policy iteration**, these two processes alternate, each completing before the other begins, but this is **not necessary**. 
* In **value iteration** only a single iteration of policy evaluation is performed in between each policy improvement.
* **Generalized Policy Iteration (GPI)** refers to the general idea of letting **policy-evaluation** and **policy-improvement** processes interact, independent of the granularity and other details of the two processes.
* Almost all RL methods are well described as GPI, i.e., all have identifiable **policies** and **value functions**, with the policy always being improved with respect to the value function and the value function always being driven toward the value function for the policy (as suggested by the above diagram).
* If both the **evaluation** process and the **improvement** process stabilize (i.e., no longer produce changes), then the value function and policy must be **optimal**.
  - The value function stabilizes only when it is consistent with the current policy, and the policy stabilizes only when it is greedy with respect to the current value function.
  - Thus, both processes stabilize only when a policy has been found that is greedy with respect to its own evaluation function.

---
## Ch 5. Monte Carlo Methods
* Our first learning methods for **estimating value functions** and **discovering optimal policies**.
* Unlike DP, here we do not assume **complete knowledge of the environment**.
* **Monte Carlo (MC)** methods require only **experience**, i.e., sample sequences of states, actions, and rewards from **actual** or **simulated** interaction with an environment.
  - Learning from **actual experience** is striking because it requires **no prior knowledge** of the environment's dynamics, yet can still attain optimal behavior. 
  - Learning from **simulated experience** requires a **model**, however the model needs **only generate sample transitions**, not the complete probability distributions of all possible transitions that is required for dynamic programming (DP). 
* To ensure that **well-defined returns** are available, here we define MC methods only for **episodic tasks**.
  - An experience is divided into episodes.
  - All episodes eventually terminate no matter what actions are selected.
  - Only on the completion of an episode the value estimates and policies are changed.
* MC methods can thus be incremental in an **episode-by-episode sense**, but not in a step-by-step (online) sense.
* MC methods sample and average returns for each state-action pair much like the **bandit** methods.
  - The main difference is that now there are **multiple states**, each acting like a different bandit problem.
  - That is, the return after taking an action in one state depends on the actions taken in later states in the same episode. 

### Monte Carlo Prediction
* We consider MC methods for learning the **state-value function** for a **given policy**, i.e., $v_{\pi}(s)$.
* Recall that the **value of a state** is the **expected return** (expected cumulative future discounted reward) starting from that state.
* One way to estimate it from **experience** is to **average** the **returns** observed after visits to that **state**. This idea underlies all **MC methods**.
* Each occurrence of state s in an episode is called a **visit** to $s$.
  - A state $s$ may be visited multiple times in the same episode.
* We call the first time a state $s$ is visited in an episode the **first visit** to $s$. 
* The **first-visit MC** method estimates $v_{\pi}(s)$ as the average of the returns following **first visits** to $s$.
* The **every-visit MC** method averages the returns following **all visits** to $s$.
* First-visit MC is shown in procedural form in the box. Every-visit MC would be the same except without the check for $S_t$ having occurred earlier in the episode.
* Both first-visit MC and every-visit MC converge to $v_{\pi}(s)$ as the number of visits (or first visits) to $s$ goes to infinity.
<img src="rl_figs/rl_first_visit_mc.png" width="700">

### Monte Carlo Estimation of Action Values
* If a **model is not available**, then it is useful to estimate **action-values** rather than **state-values**.
* **With a model**, **state-values** alone are **sufficient** to determine a **policy**.
  - One simply looks ahead **one step** and chooses whichever **action** leads to the best combination of **reward** and **next state** (as in DP).
* **Without a model**, state-values alone are **not sufficient**.
  - One must explicitly **estimate** the **value of each action** in order for the values to be useful in suggesting a policy.
  - One of goals for MC methods is to estimate $q_*$ (to achieve this, we consider the **policy evaluation** problem for **action-values**).
* The **policy evaluation** problem for **action-values** is to estimate $q_{\pi}(s, a)$, i.e., the expected return when starting in state $s$, taking action $a$, and thereafter following policy $\pi$.
* A state-action pair $(s, a)$ is said to be **visited** in **an episode**, if ever the state $s$ is visited and action $a$ is taken in it.
* The **first-visit MC** method averages the returns following the first time in each episode that the state was visited and the action was selected.
* The **every-visit MC** method estimates the value of a state-action pair as the average of the returns that have followed all the visits to it. 
* The only **complication** is that many state-action pairs **may never be visited**.
  - This is a serious problem because the purpose of learning action values is to help in choosing among the actions available in each state.
  - To compare alternatives we need to estimate the value of all the actions from each state, not just the one we currently favor.
  - This is the general problem of maintaining exploration, as discussed in the k-armed bandit problem.
* For **policy evaluation** to work for **action-values**, we must assure **continual exploration**.
  - The episodes start in a state-action pair, and that every pair has a **nonzero probability** of being selected as the start.
  - This assumption is called **exploring starts**.
  - An alternative approach to assuring that all state-action pairs are encountered is to consider only policies that are stochastic with a nonzero probability of selecting all actions in each state.

### Monte Carlo Control
* How MC estimation can **approximate optimal policies**?
* The overall idea is to proceed according to the idea of generalized policy iteration (GPI).
  - In GPI one maintains both an **approximate policy** and an **approximate value function**.
<img src="rl_figs/rl_gpi_action.png" width="200">
<img src="rl_figs/rl_policy_iter3.png" width="500">

* $E$ denotes a **complete policy evaluation**, and $I$ denotes a **complete policy improvement**.
* **Policy evaluation** is done exactly as described in DP.
* **Policy improvement** is done by making the policy greedy with respect to the current value function.
  - In this case we have an **action-value function**, and therefore **no model is needed** to construct the greedy policy. 
  - For any action-value function $q$, the corresponding greedy policy is the one that, for each $s \in S$, deterministically chooses an action with maximal action-value: $\pi(s) = \arg\max_a q(s, a)$.
* For the moment, we assume that we observe an **infinite number of episodes** and the episodes are generated with **exploring starts**.
* Here, we made two unlikely assumptions to obtain the guarantee of convergence for the MC method.
  1. The episodes have **exploring starts**.
  2. The **policy evaluation** could be done with an **infinite number of episodes**.
* To obtain a practical algorithm we will have to remove both assumptions.
* There are two ways to solve the second assumption (inifinite number of episodes):
  1. To hold firm to the idea of approximating $q_{\pi_k}$ in each policy evaluation.
  2. To give up trying to complete policy evaluation before returning to policy improvement. On each evaluation step we move the value function **toward** $q_{\pi_k}$, but we do not expect to actually get close except over many steps. One extreme form of the idea is **value iteration**, in which only one iteration of iterative policy evaluation is performed between each step of policy improvement. 
* A complete algorithm of **Monte Carlo with Exploring Starts (ES)** is given below.
<img src="rl_figs/rl_mc_es.png" width="700">

### Monte Carlo Control without Exploring Starts
* How can we avoid the unlikely assumption of **exploring starts**?
* The only general way to ensure that **all actions are selected** infinitely often is for the agent to **continue to select them**.
* There are two approaches to ensuring this: **on-policy** methods and **off-policy** methods
* **On-policy** methods attempt to evaluate or improve the **policy that is used to make decisions**.
* **Off-policy** methods evaluate or improve a **policy diffrent from that used to generate the data**.
* The **MC ES** method is an example of an **on-policy** method. 
* In **on-policy** methods the policy is generally **soft**, meaning that $\pi(a|s) > 0$ for all $s \in S$ and all $a \in A(s)$, but **gradually** shifted closer and closer to a **deterministic optimal policy**.
* The on-policy method in this section uses **$\varepsilon$-greedy policies**: most of the time they choose an **action** that has **maximal estimated action-value**, but with **probability $\varepsilon$** they instead select an action at **random**.
  - The probability of selection of **nongreedy actions**: $\frac{\varepsilon}{|A(s)|}$
  - The probability of selection of **greedy actions**: $1 - \varepsilon + \frac{\varepsilon}{|A(s)|}$
* The **$\varepsilon$-soft policies** are the policies for which $\pi(a|s) \ge \frac{\varepsilon}{|A(s)|}$, for all states and actions, for some $\epsilon > 0$. 
  - The $\varepsilon$-greedy policies are examples of $\varepsilon$-soft policies.
* The overall idea of **on-policy** MC control is that of GPI.
  - As in **MC ES**, we use **first-visit** MC methods to **estimate the action-value** function for the current policy.
  - Without the assumption of **exploring starts**, however, we cannot simply improve the policy by making it greedy with respect to the current value function, because that would **prevent further exploration** of nongreedy actions. 
  - Fortunately, GPI does not require that the policy be taken all the way to a greedy policy, only that it be moved toward a greedy policy. In our on-policy method we will move it only to an $\varepsilon$-greedy policy.
<img src="rl_figs/rl_onpolicy_mc.png" width="700">

### Off-policy Prediction via Importance Sampling
* How can control methods learn about the **optimal policy** while behaving according to an **exploratory policy**?
  - The **on-policy** approach in the preceding section learns action values not for the optimal policy, but for a near-optimal policy that still explores.
* A more straightforward approach is to use **two policies**:
  - The **target policy**: a policy that **is learned** about and that becomes the **optimal policy**.
  - The **behavior policy**: a policy that **is exploratory** and is used to **generate behavior**.
* In this case we say that learning is from data **off** the target policy, and the overall process is termed **off-policy** learning.
- **On-policy** methods are generally simpler, and **off-policy** methods are often of greater variance and are slower to converge, but on the other hand are more powerful and general.
* In this section we study off-policy methods by considering the **prediction problem**, in which both **target and behavior policies are fixed**.
  - $\pi$ is the **target policy**, and $b$ is the **behavior policy**, and both policies are considered fixed and given.
  -  We want to **estimate the expected returns** under the **target policy**, but all we have are **returns** $G_t$ due to the **behavior policy**.
  - We want to estimate $v_{\pi}$ or $q_{\pi}$, but all we have are episodes following another policy $b$, where
$b \neq \pi$.
* In order to use episodes from $b$ to estimate values for $\pi$, we require that every action taken under $\pi$ is also taken, at least occasionally, under $b$, i.e., $\pi(a|s) > 0$ implies $b(a|s) > 0$: this is called the **assumption of coverage**.
* It follows from coverage that $b$ must be **stochastic** in states where it is not identical to $\pi$, and the target policy $\pi$, on the other hand, may be **deterministic**.
  - In control, the **target policy** is typically the **deterministic greedy policy** (deterministic optimal policy) w.r.t. the current estimate of the action-value function. 
  - The **behavior policy** remains **stochastic and more exploratory**, e.g., $\varepsilon$-greedy policy.
  - Here, we consider the prediction problem, in which $\pi$ is **unchanging and given**.

#### Important Sampling
* Almost all **off-policy** methods utilize **importance sampling**, a general technique for **estimating expected values** under **one distribution given samples from another**.
* We apply importance sampling to off-policy learning by **weighting returns** according to the **relative probability of their trajectories** occurring under the **target and behavior policies**, called the **importance-sampling ratio**. 
* Given a starting state $S_t$, the **probability of the subsequent state–action trajectory**, $A_t, S_{t+1}, A_{t+1}, \cdots, S_T$, occurring under any policy $\pi$ is

$$
\begin{equation}
\begin{aligned}
Pr\{A_t, S_{t+1}, A_{t+1}, \cdots, S_T | S_t, A_t:T \sim \pi\} ={} & \pi(A_t|S_t)p(S_{t+1}|S_t,A_t)\pi(A_{t+1}|S_{t+1})\cdots p(S_T|S_{T-1},A_{T-1})\\
={} & \prod_{k=t}^{T-1}\pi(A_k|S_k)p(S_{k+1}|S_k,A_k)
\end{aligned}
\end{equation}
$$
  - $p$ is the **state-transition probability** function.
* The **relative probability of the trajectory** under the **target and behavior policies** (i.e., the **importance-sampling ratio**) is:
$$
\rho_{t:T-1} = \frac{\prod_{k=t}^{T-1}\pi(A_k|S_k)p(S_{k+1}|S_k,A_k)}{\prod_{k=t}^{T-1}b(A_k|S_k)p(S_{k+1}|S_k,A_k)} = \frac{\prod_{k=t}^{T-1}\pi(A_k|S_k)}{\prod_{k=t}^{T-1}b(A_k|S_k)}
$$
  - The **trajectory probabilities** depend on the **MDP's transition probabilities** (which are generally unknown), but they appear identical here, and thus cancel.

#### Calculating the average returns from a batch of observed episodes following policy $b$ to estimate $v_{\pi}(s)$
* Recall that we wish to **estimate the expected returns** under the **target policy** $\pi$, but all we have are **returns** $G_t$ due to the **behavior policy** $b$. 
  - These returns have the wrong expectation $\mathbb{E}[G_t|S_t=s] = v_b(s)$ and so cannot be averaged to obtain $v_{\pi}$. This is where **importance sampling** comes in.
  - The ratio $\rho_{t:T-1}$ transforms the **returns** to have the right expected value: $\mathbb{E}[\rho_{t:T-1}G_t|S_t=s] = v_{\pi}(s)$
* Here, we assume that the number time steps **increases** across episode boundaries (e.g., if the first episode of the **batch** ends in a terminal state at time 100, then the next episode begins at time $t = 101$).
* $\mathcal{T}(s)$: the set of **all time steps** in which state $s$ is visited.
* $T(t)$: the first time of termination following time $t$.
* $G_t$: the return after $t$ up through $T(t)$.
* $\{G_t\}_{t \in \mathcal{T}(s)}$: the returns that pertain to state $s$.
* $\{\rho_{t:T(t)-1}\}_{t \in \mathcal{T}(s)}$: the corresponding importance-sampling ratios.
* **Ordinary importance sampling**: when importance sampling is done as a simple average:
$$
V(s) = \frac{\sum_{t \in \mathcal{T}(s)}\rho_{t:T(t)-1}G_t}{|\mathcal{T}(s)|}
$$
* **Weighted importance sampling**: when a weighted average is used:
$$
V(s) = \frac{\sum_{t \in \mathcal{T}(s)}\rho_{t:T(t)-1}G_t}{\sum_{t \in \mathcal{T}(s)}\rho_{t:T(t)-1}}
$$

#### Incremental implementation
* Suppose we have a **sequence of returns** $G_1, G_2, \cdots, G_{n-1}$, all starting in the **same state**, and each with a corresponding **random weight** $W_i$ (e.g., $W_i = \rho_{t_i:T(t_i)-1}$).
* We wish to form the estimate $V_n$, and keep it up-to-date as we obtain a single additional return $G_n$:
$$
V_n = \frac{\sum_{k=1}^{n-1}W_kG_k}{\sum_{k=1}^{n-1}W_k}, n \ge 2
$$

* In addition to keeping track of $V_n$, we must maintain for **each state** the cumulative sum $C_n$ of the **weights** given to the **first $n$ returns** ($C_0 = 0$):
$$
C_{n+1} = C_n + W_{n+1}
$$

* The update rule for $V_n$ is:
$$
V_{n+1} = V_n + \frac{W_n}{C_n}[G_n - V_n], n \ge 1
$$

* Below is a complete episode-by-episode incremental algorithm for MC policy evaluation.
<img src="rl_figs/rl_offpolicy_mc.png" width="700">

### Off-policy Monte Carlo Control
* Now, lets present **off-policy MC control** methods.
* Recall that an advantage of this separating target and behavior policies is that the **target policy** may be **deterministic** (e.g., greedy), while the **behavior policy** can continue to **sample all possible actions**.
* Off-policy methods follow the **behavior policy**, while learning about and improving the **target policy**.
  - These techniques require that the behavior policy has a **nonzero probability** of selecting **all actions** that might be selected by the target policy (**coverage**).
  - To **explore all possibilities**, we require that the **behavior policy** be **soft** (i.e., that it select all actions in all states with nonzero probability).
* The following box shows an **off-policy MC control** method (based on GPI and weighted importance sampling) for estimating $\pi_*$ and $q_*$.
* The **target policy** $\pi \approx \pi_*$ is the greedy policy with respect to $Q$, which is an estimate of $q_{\pi}$. 
* The **behavior policy** $b$ can be anything, but in order to assure convergence of $\pi$ to the optimal policy, an infinite number of returns must be obtained for each pair of state and action (this can be assured by choosing $b$ to be $\varepsilon$-soft).
<img src="rl_figs/rl_offpolicy_mc_control.png" width="700">